# 第4章 TensorFlow数据处理

## 4.2 构建数据集的常用方法
	tf.data.Dataset表示一串元素（element），其中每个元素包含一个或多个Tensor对象。例如：在一个图像流水线（pipeline）中，一个元素可以是单个训练样本，它们带有一个表示图像数据的张量和一个标签组成的数据对（pair）。有两种不同的方式构建一个数据集，具体如下。
	直接从 Tensor 创建数据集（例如 Dataset.from_tensor_slices()）；当然 NumPy 也是可以的，TensorFlow 会自动将其转换为 Tensor。
	通过对一个或多个 tf.data.Dataset 对象的变换（例如 Dataset.batch()）来创建数据集。
这两类构建方法又可以进一步分为7种方法，如表4-1所示。

|表4-1 构建数据集的常用方法|
|--------------------------|

|数据格式	|读取方法|	备注|
|:--------------------|:-------------------------------|:--------------|
从NumPy数组读取	    |tf.data.Dataset.from_tensor_slices |当数据较小时|
从Python Generator读取	|tf.data.Dataset.from_generator	|   |
从文本数据读取	     |tf.data.TextLineDataset	|        |
从CSV数据读取	   |tf.data.experimental.CsvDataset|	|
从TFRecord data读取  |tf.data.TFRecordDataset 	|TFRecord 是TensorFlow中自带的，它是一种方便储存比较大的数据集的数据格式(二进制格式)，当内存不足时，我们可以将数据集制作成TFRecord格式的再将其解压读取。|
从二进制文件读取数据 |tf.data.FixedLengthRecordDataset|	|
从文件集中读取数据   |tf.data.Dataset.list_files()|	|


### 4.2.1 从内存中读取数据
	从内存中读取数据的方法适用于数据较少，可直接存储于内存中的情况，其主要包括tf.data.Datasets.from_tensor_slices方法和tf.data.Datasets..from_generator（从生成器读取）方法。它从内存中读取数据，输入参数可以是NumPy的多维数组，也可以是TensorFlow的张量，还可以是Python的列表(list)、元祖以及字典等。  
1.从NumPy中读取数据  
如果输入数据为NumPy或tf.Tensor，可使用 Dataset.from_tensor_slices()读取。

In [1]:
import tensorflow as tf
import numpy as np

# 从NumPy数组创建数据集
dataset1 = tf.data.Dataset.from_tensor_slices(np.arange(10))   
print(dataset1)
# <TensorSliceDataset shapes: (), types: tf.int64>

#从张量中读取数据
dataset2 = tf.data.Dataset.from_tensor_slices(tf.constant([1,2,3,4]))
print(dataset2)
# <TensorSliceDataset shapes: (), types: tf.int64>

<TensorSliceDataset shapes: (), types: tf.int32>
<TensorSliceDataset shapes: (), types: tf.int32>


2. 从迭代器中读取数据  
可以读取NumPy、张量数据，也可以读取迭代器中的数据。迭代器中的数据虽然也在内存中，但所耗的资源较小。在使用tf.data.Dataset.from_generator()方法构建数据集时，我们需要提供3个参数（generator、output_types、output_shapes），其中generator参数必须支持iter()协议，也就是generator需要具有迭代功能，推荐使用Python yield。

In [2]:
import numpy as np
import tensorflow as tf

#定义数据生成器函数 
def data_generator():
    dataset = np.array(range(5))
    for d in dataset:
        yield d
 
ds = tf.data.Dataset.from_generator(data_generator, (tf.int32), (tf.TensorShape([])))
#查看数据
for i in ds:
    print(i.numpy(),end=",")


0,1,2,3,4,

### 4.2.2 从文本中读取数据
	内存数据一般较小，很多情况下，我们需要加载的数据保存在文本文件中，比如日志文件。tf.data.TextLineDataset可以帮助我们从文本文件中读取数据，源文件中（比如日志信息）的一行代表一个样本。 tf.data.TextLineDataset 的方法签名为：tf.data.TextLineDataset(filenames, compression_type=None, buffer_size=None, num_parallel_reads=None) ，其中：
- filenames ：一系列将要读取的文本文件的路径+名字。
- compression_type ：文件的压缩格式，tf.data.TextLineDataset可以从压缩文件中直接读取数据或者将数据写入压缩文件中以节省磁盘空间。它的默认值是None，表示不压缩。
- buffer_size ：一次读取的字节数量，如果不指定，则将由TensorFlow根据一定策略选择。
- num_parallel_reads ：如果有多个文件需要读取，用num_parallel_reads可以指定同时读取文件的数量。默认情况下按照文件顺序一个一个读取。
 为了演示通过TextLineDataset读取数据的过程，这里以一个数据文件（test.txt）为例。

In [3]:
import tensorflow as tf

ds_txt = tf.data.TextLineDataset(filenames = ["./data/test.txt"] ).skip(1) # 忽略掉第一行,第一行为标题
for line in ds_txt.take(2):
    print(line)

tf.Tensor(b'Suzhou, JiangSu, 1', shape=(), dtype=string)
tf.Tensor(b'Wuxi, JiangSu, 0', shape=(), dtype=string)


实际上，csv文件也是一个文本文件，TextLineDataset也可以读取csv文件的数据，但是根据上面的例子我们知道，tf.data.TextLineDataset会将文件中的每一行数据读成一个张量，如果我们想保留csv文件中数据的结构（比如第一列代表了什么，第二列代表了什么），我们可以使用tf.data.experimental.make_csv_dataset 这个函数。这里使用泰坦尼克生存预测数据为例，第一行为标题。

In [4]:
import tensorflow as tf 

ds4 = tf.data.experimental.make_csv_dataset(
      file_pattern = ["./data/train.csv"],
      batch_size=2, 
      na_value="",
      ignore_errors=True)

for data in ds4.take(2):
    print(data)

OrderedDict([('survived', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 0])>), ('sex', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'male', b'male'], dtype=object)>), ('age', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40., 27.], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 0])>), ('parch', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([0, 2])>), ('fare', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 27.7208, 211.5   ], dtype=float32)>), ('class', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'First', b'First'], dtype=object)>), ('deck', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'unknown', b'C'], dtype=object)>), ('embark_town', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'Cherbourg', b'Cherbourg'], dtype=object)>), ('alone', <tf.Tensor: shape=(2,), dtype=string, numpy=array([b'y', b'n'], dtype=object)>)])
OrderedDict([('survived', <tf.Tensor: shape=(2,), dtype=int32, numpy=array([1

此外，可以指定在读取数据时只读取我们感兴趣的列，通过传入参数select_columns 来实现。
	在train.csv文件中，survived列是标签数据。可以通过指定label_name来分离样本数据和标签数据。

### 4.2.3 读取TFrecord格式文件
	TFRecord 是 TensorFlow 自带的一种数据格式，是一种二进制文件。它是TensorFlow 官方推荐的数据保存格式，其数据的存储、读取操作更加高效。具体来说，TFRecord的优势可概括为：  
1）支持多种数据格式；  
2）更好的利用内存，方便复制和移动；  
3）将二进制数据和标签(label)存储在同一个文件中。

TFRecord 格式文件的存储形式会很合理地帮我们存储数据。TFRecord 内部使用了 Protocol Buffer 二进制数据编码方案，它只占用一个内存块，只需要一次性加载一个二进制文件的方式即可，简单，快速，尤其对大型训练数据很友好。当我们的训练数据量比较大的时候，TFRecord可以将数据分成多个 TFRecord 文件，以提高处理效率。  
	假设有一万张图像， TFRecord 可以将其保存成 5 个.tfrecords 文件（具体保存成几个文件，要看文件大小），这样我们在读取数据时，只需要进行5 次数据读取。如果把这一万张图像保存为NumPy格式数据，则需要进行10000次数据读取。  
	我们可以使用tf.data.TFRecordDataset类读取TFRecord文件。

In [5]:
import os

input_path="./data/"
ds = tf.data.TFRecordDataset([os.path.join(input_path, 'records')])
# 如果需要读取压缩文件，值需要指定compression_type
# ds = tf.data.TFRecordDataset([os.path.join(output_dir, 'records')], compression_type='GZIP')
print(ds)

<TFRecordDatasetV2 shapes: (), types: tf.string>


TFRecord格式非常高效，接下来我们将详细介绍如何把自己的数据转换为TFRecord格式的数据，以及转换的具体步骤等内容。